In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
import warnings

## Data Visualization

In [ ]:
df = pd.read_csv('C:\\Users\\alvar\\Documents\\tf_templates\\DATA\\RSCCASN.csv', index_col='DATE', parse_dates=True)
df.head()

In [ ]:
df.columns = ['Sales']

In [ ]:
df.plot(figsize=(12,6));

## Train Test Split

In [ ]:
test_size = 18 # data is monthly, 18 = 1.5 years
test_ind = len(df) - test_size

In [ ]:
train = df.iloc[:test_ind]
test = df.iloc[test_ind:]

## Scale Data

In [ ]:
scaler = MinMaxScaler()

scaled_train = scaler.fit_transform(train)
scaled_test = scaler.transform(test)

## Time Series Generator

In [ ]:
length = 12
batch_size = 1
generator = TimeseriesGenerator(scaled_train, scaled_train, length=length, batch_size=batch_size)
validation_generator = TimeseriesGenerator(scaled_test, scaled_test, length=length, batch_size=batch_size)

## LSTM Model and Training

In [ ]:
n_features = 1

In [ ]:
model = Sequential()

model.add(Input(shape=(length, n_features)))

model.add(LSTM(units=100, activation='relu'))
model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam', loss='mse')

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
warnings.filterwarnings('ignore')

model.fit(generator, epochs=15, validation_data=validation_generator, callbacks=[early_stop])

## Evaluation

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
losses[['loss','val_loss']].plot();

In [ ]:
test_predictions = []

first_eval_batch = scaled_train[-length:]
current_batch = first_eval_batch.reshape((1, length, n_features))

for i in range(len(test)):
    
    # get prediction 1 time stamp ahead ([0] is for grabbing just the number instead of [array])
    current_pred = model.predict(current_batch)[0]
    
    # store prediction
    test_predictions.append(current_pred) 
    
    # update batch to now include prediction and drop first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
predictions = scaler.inverse_transform(test_predictions)
test['Predictions'] = predictions

In [ ]:
test.plot();

## Forecasting

In [ ]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

In [ ]:
length = 12
batch_size = 1
generator = TimeseriesGenerator(scaled_data, scaled_data, length=length, batch_size=batch_size)

In [ ]:
model = Sequential()

model.add(Input(shape=(length, n_features)))

model.add(LSTM(units=100, activation='relu'))
model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam', loss='mse')

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)

In [ ]:
model.fit(generator, epochs=20, validation_data=validation_generator, callbacks=[early_stop])

In [ ]:
forecast = []
# Replace periods with whatever forecast length you want
periods = 12

first_eval_batch = scaled_data[-length:]
current_batch = first_eval_batch.reshape((1, length, n_features))

for i in range(periods):
    
    # get prediction 1 time stamp ahead ([0] is for grabbing just the number instead of [array])
    current_pred = model.predict(current_batch)[0]
    
    # store prediction
    forecast.append(current_pred) 
    
    # update batch to now include prediction and drop first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
forecast = scaler.inverse_transform(forecast)

In [ ]:
forecast

In [ ]:
df

In [ ]:
forecast_index = pd.date_range(start='2019-11-01', periods=periods, freq='MS')

In [ ]:
forecast_df = pd.DataFrame(data=forecast, index=forecast_index, columns=['Forecast'])

In [ ]:
df.plot()
forecast_df.plot();

In [ ]:
forecast_df.plot(ax=df.plot(),figsize=(12,6));
plt.xlim('2000-01-01', '2020-11-01')